<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear Op Amp 01 : Open Loop

This project starts working with the Operational Amplifier in **open loop** operation.

Version 1.2 (12/3/2019)  
License information is at the end of the document

---
**Bill Of Materials (BOM):**

* Dual Opamp MCP6002
* $1k\Omega$ resistor

---

## Meet the Opamp

The **operational amplifier**, opamp for short, is a device that amplifies the **voltage difference** Vd between two input nodes: A positive (+) or **non inverting input** (nii) and a negative (-) or **inverting input** (ii). The output of the amplifier Vo is **A** times the input voltage difference.
The following figure shows the operational amplifier symbol together with the zero order basic model that implements the above explanation:

![fig 01](images\L_OA_01\fig01.png)

The model shows that the output node is forced, by a dependent voltage source, to a voltage that is proportional to the Vd voltage between the inputs. 
This is a model, an idealization of what the opamp should be. We will see later that real opamps fail to operate always as this model.
Observe that the input terminals are not connected to anything. The described model supposes that that the current entering the opamp through its input terminals is zero.
Observe also that the basic model includes a node, the ground node, which is not included in the opamp symbol. 

From the above model we can get the output of the opamp as:

$$V_O = A \cdot V_d = A\left(V_{(+)}-V_{(-)}\right)$$

The greater the A value, the better. An ideal opamp will operate just like the above model with an infinite A gain. So it will satisfy:

$$V_O = A\left(V_{(+)}-V_{(-)}\right) \quad with \quad A \rightarrow \infty$$

This is de **ideal** opamp operation. That’s what any opamp aspires to be.
 
We cannot get our hands on an ideal opamp, we can only buy real ones. In this document we will work with the Microchip **MCP6002** integrated circuit in the 8 pin **PDIP** package version. This component features two opamps on a single chip:

![mcp6002](images\L_OA_01\mcp6002.png)

If the two included opamps were ideal, we wouldn’t need any more information as they would just behave as the model says. If they were not ideal but they worked as the zero order model with a finite gain, we would only need the gain A value. The less ideal a component is, the more information we need about it to know how it really operates.

All the needed information about a component operation can be obtained from the component datasheet that the manufacturer releases free of charge. In order to know how our MCP6002 will operate, we need its datasheet.

Obtain the MCP6002 datasheet. Just google on **"MCP6002 datasheet"**. It will probably be the first PDF result.

The datasheet shows all the ways that the opamp doesn’t behave ideally. And it has 40 pages! For instance, we can see the finite gain, defined as Open Loop Gain (AOL).

![aol](images\L_OA_01\aol.png)

First thing we see is that we don’t know the AOL value. The datasheet says that the typical value is 112 dB. But the typical value is just the normal value you should expect. A particular opamp can have a different value, as low as the minimum limit of 88 dB. Observe that there is no maximum value. It is unbound. That means that you can get any gain over 88 dB. 

Gain is provided in dB. Remember that A\[dB\], relates to linear gain A as:

$$A[dB] = 20 \cdot log_{10}(A)$$

Calculate the typical and minimum linear gain A values for the MCP6002. This gain is usually given unitless (V/V) or as V/mV.

You can fill-in the formulas in the **code cell** below to do the calculations

In [ ]:
Aol_typ = 
Aol_min = 
print('Typical Aol is ',Aol_typ)
print('Minimum Aol is ',Aol_min)

Enough calculations for now, let’s see how this device behaves.

---

![Practical Icon](images/pt.png)

---

## Connecting the board

Before performing any measurement we need to connect to the board. This also requires setting the proper paths for the **SLab** data files.

In [ ]:
# Import the SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

If something goes wrong with the board you can always **reset** it, and run again the second cell to reconnect with the board. You don't need to repeat the **import** if you don't close this notebook.

Before starting with the opamp circuits it is a good idea to check the board calibration. This is specially easy if the board wires are in the **SLab Standard Docking** position.

In [ ]:
# Check the calibration for the first four ADCs
slab.checkCalibration(pause=False,na=4)

## Powering the Opamp

First thing to note is that the **MCP6002** component has more terminals than the ideal one. The first order opamp model didn’t feature any Vdd terminal, but real opamps need to get power from somewhere. Regarding the supply, we will power the opamp from the SLab hardware board Vdd supply and we need to check that we won’t damage the opamp.
The opamp datasheet, in the **Absolute Maximum Ratings** table shows the maximum allowed voltage difference between the positive and negative supply nodes:

![amr](images\L_OA_01\amr.png)

That means that using a supply over 7V will **damage** the opamp. Never do that. The fact that the opamp is not damaged doesn’t mean that it will work as expected. Going down the datasheet se can see the supply requirements:

![Supply Requirements](images\L_OA_01\supply_req.png)

As we see we can operate the opamp with supply voltages as low as 1.8 V and as high as 6.0 V. Any voltage between 0 V and 1.8V or between 6.0 V and 7.0 V won’t damage the opamp but the device also won’t operate as expected. 
The table also shows us the quiescent current IQ. This is the current needed just to operate without taking into account any current provided on the device outputs. We see that the MCP6002 chip can draw up to $170 \mu A$ without doing anything, just by being connected to the supply.

We can check the IQ ourselves.

A $170 \mu A$ will produce a voltage drop of 170 mV on a $1 k\Omega$ resistor. The following circuit enables us to measure this value. Remember that the MCP6002 features two opamps. We connect the inputs of both to the opamp negative supply so that the noise on both opamps is not amplified.

![fig02](images\L_OA_01\fig02.png)

Next to each terminal, a small number indicates the related pin in the integrated circuit.

---

**BUILD TASK**

Build the circuit on the breadboard.
Add also the ADC, Vdd and GND connections.
    
---    
    
To obtain the current we only need to build the circuit and request the current on the resistor on a Python console. First, we import the slab module, then we connect to the hardware board and finally we request the current on a $1000\Omega$ resistor connected between ADC1 output and ground.    

In [ ]:
print('Iq = ',1e6*slab.rCurrent(1000,1),'uA')

Compare the measured Iq value with the datasheet values. How does it compare? Is it far from the typical value? 

If you are curious, you can check how this current relates to the supply voltage. If you don’t want to hassle with that, just skip to the next section.

We will connect the opamp positive supply to **DAC1** output instead of Vdd so that we can control this voltage. Remember to disconnect previously **Vdd** as you should never connect together the outputs of two supplies.

![fig03](images\L_OA_01\fig03.png)

As we know, the opamp should work only from 1.8V, but, as we won’t damage it below that value, we will sweep all DAC 1 voltages up to 3.2 V in 0.1 V steps.

>`data = slab.dcSweep(1,0,3.2,0.1)`

The **dcSweep** command returns a five element tuple. Element 0 is the DAC 1 value and elements 1 to 4 are ADC values. You can ask for help on the command if you want this information:

>`slab.help('dcSweep')`

We want to give the current in $\mu A$, so you need to manage the units:

$\qquad I_Q[\mu A] = 10^6 I_Q[A] = 10^6 \frac{V_{ADC1}[V]}{R1[\Omega]}
= 10^6 \frac{V_{ADC1}[V]}{1000\Omega} = \frac {1000}{1\Omega}V_{ADC1}[V]$

Now we can compute supply voltage and current. 

>`Vsup = data[0] – data[1]`  
>`Iq = data[1]*1000`

Then we can plot the information using the **plot11** command:

>`slab.plot11(Vsup,Iq,'Iq Current','Vsup(V)','Iq(uA)')`

All this code is in the cell below:

In [ ]:
data = slab.dcSweep(1,0,3.2,0.1)
Vsup = data[0] - data[1]
Iq = data[1]*1000
slab.plot11(Vsup,Iq,'Iq Current','Vsup(V)','Iq(uA)')

You can compare the results with the data on the opamp datasheet:

Compare the measurements with the datasheet curves.  
How does the $Iq$ dependence with supply voltage compare?

## Open loop circuit

After checking the quiescent current we can now see the basic behavior to see how it relates with the first order model. The following circuit will enable us to obtain the response of the opamp.

![fig04](images\L_OA_01\fig04.png)

If you prefer, the following schematic is equivalent and makes explicit that **Vdd**, **DAC1** and **DAC2** voltages are forced and **ADC1** is only measured.

![fig05](images\L_OA_01\fig05.png)

**Build the circuit of the above figure**    

We can check the output of the amplifier at ADC 1 as function of the difference voltage set between DAC 1 and DAC 2. First, we set DAC 2 at 1V. So that V(-) = 1V.

>`slab.setVoltage(2,1)`

Now we can obtain the DAC 1 to ADC 1 response for DAC 1 values between 0V and 2V in 50mV steps by using the curveVV command. This command plots the ADC 1 voltage as function of the DAC 1 voltage. The curveVV command is not contained in the main slab.py file. It is part of the DC submodule, so we will need to import it. For ease of use we will use an alias dc for the slab_dc module namespace.

>`import slab_dc as dc`  
>`dc.curveVV(0,2,0.05)`

The curve doesn’t give the opamp input Vd voltage but the V(+) voltage. Using an ideal opamp, we should get:

$\qquad V_{ADC1}(V_{DAC1}) = A_{OL} \left( V_{(+)}-1V \right)$

The Vo(Vi) curve should be a line that pass through the Vi = 1V, Vo = 0V point with an AOL slope. If you want you can compute the Vo(Vd) curve using a slightly more complex alternative method:

The **dcSweep** command returns a 5 element tuple with the DAC 1 voltage as element 0 and the ADC voltages as elements 1, 2, 3  and 4.

The following code cells obtains the $V_O(V_d)$ opamp response using this command.

In [ ]:
slab.setVoltage(2,1)
data = slab.dcSweep(1,0,2,0.05)
vd = data[0] - 1
vo = data[1]
slab.plot11(vd,vo,'Open Loop DC response','Vd(V)','Vo(V)')

Compare the measurements with the expected result.  
Does the Opamp work as expected?  
    
You should observe a big difference from the model prediction. The output does not always behave as an $A_{OL}$ slope line. It saturates at two levels $V_{O \:max}$ and $V_{O \:min}$.

![fig06](images\L_OA_01\fig06.png)

This deviation from the ideal operation is also included in the manufacturer datasheet as the Maximum Output Voltage Swing parameter.

![vol & voh](images\L_OA_01\vol_voh.png)

We see that the output can go as low as 25 mV over the negative Vss supply (GND in our case) and as high as 25 mV below the positive Vdd voltage. No **real** operational amplifier can output voltages outside of the supply range so, in this regard, the MCP6002, although not ideal, is quite good. An opamp, like the MCP6002, that is capable to drive its output very close to the supply limits is qualified as **Rail to Rail Output** or **Full Rail Output**.

If you can get your hands on a **TLC272** dual opamp, for instance, you can check that $V_{O \:min}$ can nearly reach the negative supply but $V_{O \:max}$ gets only to about 1V below the positive supply.

So, a real opamp never behaves like ideal or the basic zero order model because it saturates. A model more akin to the real operation can use the same circuit as the zero order case but the equations shall now be written as:

$\qquad V_d = V_{(+)} - V_{(-)}$

$\qquad V_O = V_{O \: max} \qquad if \quad A \cdot V_d \geq V_{O\: max}$

$\qquad V_O = A \cdot V_d \qquad if \quad V_{O\: min} \leq A \cdot V_d \leq V_{O\: max}$

$\qquad V_O = V_{O\: min} \qquad if \quad A \cdot V_d \leq V_{O\: min}$

See how as we depart from ideality more parameters creep in. An ideal opamp had no parameter at all (as gain was always infinite). Zero order model had one gain A parameter. Now we have, at least three parameters: $A$, $V_{O \:max}$ and $V_{O \:min}$. Four if you count quiescent current $I_Q$.

## Gain Troubles

Obtaining the gain can be tricky. It seems easy because you only need to get two points to calculate gain from them:

$\qquad A_{OL} = \frac{V_{O2}-V_{O1}}{V_{d2}-V_{d1}}$

But you need to be sure that those points correspond to real measurement out of the saturation region. Let’s say you have an opamp with the response shown in red and you get the blue measurement points:

![fig07](images\L_OA_01\fig07.png)

When you obtain the plot, the program will join the dots to show you the following blue curve:

![fig08](images\L_OA_01\fig08.png)

As we remember, in the previous measurements, we used 50mV spacing between DAC 1 points, so we cannot guarantee that two points are in the non saturated region if they are not 50mV or more afar from the closer saturated point. 
In general you need to use a $\Delta V$ voltage step low enough so that the two used measurement points are, at least, a distance $\Delta V$ or more from the saturated regions.

![fig09](images\L_OA_01\fig09.png)

The SLab system is limited in how small a DAC step can be. Remember that a n bit DAC output voltage VDAC relates to its digital input nDAC with the formula:

$\qquad V_{DAC} = \frac{V_{dd}}{2^n}n_{DAC}$

So the resolution will be one count change:

$\qquad \Delta V_{DAC} = \frac{V_{dd}}{2^n}$

Note that this is the resolution. It gives the minimum possible DAC change. It is not the precission that relates to the uncertainty of the value we set. That later value can be much greater and relates to how well the board calibration was performed.

If you are using a **F303RE Zero Board**, the **DAC** resolutions are 12 bit. For any other hardware board or buffering circuit check its related information.

As we know the typical $A_{OL}$ value for the MCP6002, we can check the resolution requirements we have. If we suppose, in a fortunate case, that we can get two points separated 1/3 the output range of the opamp, powered at 3.3V, we can calculate the needed resolution:

$\qquad A_{OL} = \frac{\Delta V_O}{\Delta V_d} = \frac{V_{dd}/3}{\Delta V_{DAC 1}}
\qquad \Delta V_{DAC1} = \frac{V_{dd}}{3 \cdot A_{OL}}$

---

**THEORETICAL TASK**  
Obtain the required DAC 1 resolution and the actual resolution we have in the SLAB system.  
Are we expected to be able to directly measure the AOL gain?  
Which is the maximum gain we can measure?  
 
---   
   
You can fill-in the formulas in the code cell below to do the calculations

In [ ]:
Resolution = 
Max_Gain = 
print('DAC 1 resolution shall be',Resolution,'V')
print('Maximum gain we can measure is',Max_Gain)

## Last Comments

In this document we have performed several measurements in an open loop opamp. We call the circuit open loop because the output never loops to the input. That is, there is no flux of information that goes from the output and is feedback on the input.

An open loop configuration is never useful for amplification because amplification is too high for practical purposes. Moreover, we don’t really know what the amplification is because it can broadly change from chip to chip and, in the same chip, due to drift or ambient effects like the chip temperature.

Open loop configuration can be useful, however, for nonlinear usage. As the gain is so high, the opamp works like a comparator on the input Vd voltage. Vo is Vomax if Vd is positive and Vo is Vomin if Vd is positive.

From now on, in order to implement linear functionalities like amplification, we will use the opamp in closed loop. That requires to feedback information from the output back to the input.


## References

**SLab Python References**   
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module.  
They should be available in the SLab/Doc folder.

**TinyCad**  
Circuit images on this document have been drawn using the free software TinyCad  
https://sourceforge.net/projects/tinycad/

**Matplotlib**  
All the functions plots have been generated using the Matplotlib SciPy package.  
https://matplotlib.org/

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">